In [1]:
import numpy as np
import os
import gzip
import random
import time

import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from io import BytesIO

In [6]:
data = ""
ddir = "../data/books/witcher_rus/"
for fn in sorted(os.listdir(ddir)):
    if fn.endswith(".gz"):
        with gzip.open(ddir + fn, "rb") as f:
            data += f.read().decode("utf-8").replace("\r", "")
print(sorted(set(data)))
print("%s chars" % len(data))

['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё']
5088376 chars


Network schematics:
```
         +-<-[bh|Whh]-<-+-{h}
         |              |
-[Wxh]->-+->--[tanh]-->-+->-[Why|by]-
```

In [5]:
class Net:
    def __init__(self, sx, sy, sz):
        self.sizes = (sx, sy, sz)
        opts = {"dtype": np.float32}
        
        self.Wxh = np.zeros((sx, sh), **opts)
        self.Whh = np.zeros((sh, sh), **opts)
        self.Why = np.zeros((sh, sy), **opts)
        
        self.bh = np.zeros(sh, **opts)
        self.by = np.zeros(sy, **opts)
        
        self.h = np.zeros(sh, **opts)
    
    def randomize(self, mag):
        for k in ["Wxh", "Whh", "Why", "bh", "by"]:
            setattr(self, k, mag*np.random.randn(*getattr(self, k).shape))

    def __iter__(self):
        return iter([self.Wxh, self.Whh, self.Why, self.bh, self.by])

    def zcopy(self):
        return Net(*self.sizes)

    def mkmem(self):
        return [[np.zeros_like(b) for _ in range(2)] for b in self.biases]

    def fill(self, value):
        for v in self:
            v.fill(value)

In [6]:
def act(x):
    return np.tanh(x)

def act_deriv(x):
        return 1/np.cosh(x)**2

def cost(a, y):
    d = a - y
    return np.dot(d, d)/2

def cost_deriv(a, y):
    return a - y

In [7]:
def feedforward(net, x, fmem=False):
    a = x + net.biases[0]
    if fmem:
        mem = net.mkmem()
        mem[0][0] = a
    else:
        mem = None

    for i, (w, b) in enumerate(zip(net.weights, net.biases[1:])):
        a = act(a)
        if fmem:
            mem[i][1] = a

        a = np.dot(w, a) + b
        if fmem:
            mem[i + 1][0] = a

    return (a, mem)

In [8]:
def backprop(net, a, y, mem, neterr):
    e = cost_deriv(a, y)
    zmem = list(zip(*mem))
    zlist = list(zip(
        net.weights, net.biases[1:],
        neterr.weights, neterr.biases[1:],
        zmem[0][1:], zmem[1][:-1]
    ))
    last = True
    for w, b, ew, eb, v, ap in reversed(zlist):
        if not last:
            e = e*act_deriv(v)
        eb += e
        ew += np.outer(e, ap)
        e = np.dot(e.transpose(), w)
        last = False
    neterr.biases[0] += e

In [9]:
def learn(net, neterr, rate, adagrad=None):
    if adagrad is None:
        for v, ev in zip(net, neterr):
            v -= rate*ev
    else:
        for v, ev, av in zip(net, neterr, adagrad):
            av += ev**2
            v -= rate*(ev/np.sqrt(av + 1e-8))

In [10]:
net = Net((imgsize[0]*imgsize[1], 30, 10), mag=1e-2)
neterr = net.zcopy()
adagrad = net.zcopy()

In [11]:
batchsize = 10
rate = 3e-1
for iepoch in range(10):
    print("epoch %d:" % iepoch)
    print("train:")
    totalcost = 0.0
    tstart = time.time()
    random.shuffle(trainset)
    for batch in [trainset[p:p+batchsize] for p in range(0, len(trainset), batchsize)]:
        for digit, img in batch:
            res = np.array([i == digit for i in range(10)])
            out, netmem = feedforward(net, img, fmem=True)
            totalcost += cost(out, res)
            backprop(net, out, res, netmem, neterr)
        learn(net, neterr, rate/batchsize, adagrad)
        neterr.fill(0)
    print("cost avg: %.4f" % (totalcost/len(trainset)))
    print("time elapsed: %.2f s" % (time.time() - tstart))

    print("test:")
    totalcost = 0.0
    hitcount = 0
    tstart = time.time()
    for digit, img in testset:
        res = np.array([i == digit for i in range(10)])
        out, _ = feedforward(net, img)
        totalcost += cost(out, res)
        hitcount += digit == np.argmax(out)
    print("cost avg: %.4f" % (totalcost/len(testset)))
    print("hit count: %d / %d" % (hitcount, len(testset)))
    print("time elapsed: %.2f s" % (time.time() - tstart))

print("done")

epoch 0:
train:
cost avg: 0.0953
time elapsed: 17.27 s
test:
cost avg: 0.0747
hit count: 9292 / 10000
time elapsed: 0.48 s
epoch 1:
train:
cost avg: 0.0726
time elapsed: 16.89 s
test:
cost avg: 0.0687
hit count: 9342 / 10000
time elapsed: 0.55 s
epoch 2:
train:
cost avg: 0.0677
time elapsed: 20.24 s
test:
cost avg: 0.0656
hit count: 9375 / 10000
time elapsed: 0.45 s
epoch 3:
train:
cost avg: 0.0650
time elapsed: 20.68 s
test:
cost avg: 0.0641
hit count: 9375 / 10000
time elapsed: 1.30 s
epoch 4:
train:
cost avg: 0.0632
time elapsed: 20.32 s
test:
cost avg: 0.0632
hit count: 9380 / 10000
time elapsed: 0.52 s
epoch 5:
train:
cost avg: 0.0617
time elapsed: 15.12 s
test:
cost avg: 0.0617
hit count: 9394 / 10000
time elapsed: 0.41 s
epoch 6:
train:
cost avg: 0.0607
time elapsed: 16.59 s
test:
cost avg: 0.0614
hit count: 9386 / 10000
time elapsed: 0.47 s
epoch 7:
train:
cost avg: 0.0598
time elapsed: 16.08 s
test:
cost avg: 0.0611
hit count: 9398 / 10000
time elapsed: 0.42 s
epoch 8:
train:


In [12]:
random.shuffle(testset)
for digit, img in testset[:5]:
        out, _ = feedforward(net, img)
        display("%s: %s"  % (digit, np.argmax(out)), mkimg(img, imgsize))

'1: 1'

'0: 0'

'6: 6'

'9: 9'

'0: 0'

In [13]:
rate = 1e-1
for digit in range(10):
    img = np.random.random(net.sizes[0])*1e-2
    res = np.array([i == digit for i in range(10)])
    for i in range(1000):
        out, mem = feedforward(net, img, fmem=True)
        cv = cost(out, res)
        neterr.fill(0)
        backprop(net, out, res, mem, neterr)
        img -= rate*neterr.biases[0]
        img = np.clip(img, 0, 1)
    display(np.argmax(out), cv, mkimg(img, imgsize))

0

0.000496736855122969

1

8.595038852260855e-05

2

...

8.742895787855701e-05

8

0.0028123830821686

9

7.173522811827803e-05